In [1]:
# Load and examine the dataset
import pandas as pd
import numpy as np

# Load the parquet file
data_path = "scope_onside_common_v3.parquet"
df = pd.read_parquet(data_path)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
print(df.head())

# Check if 'smiles' column exists
if 'smiles' in df.columns:
    print(f"\nSMILES column found! Number of SMILES: {len(df['smiles'])}")
    print("Sample SMILES:")
    print(df['smiles'].head(10).tolist())
else:
    print("\nAvailable columns:", df.columns.tolist())
    print("Please check which column contains the SMILES data")

Dataset shape: (34741, 7)
Columns: ['drug_chembl_id', 'target_uniprot_id', 'label', 'smiles', 'sequence', 'molfile_3d', 'rxcui']

First few rows:
  drug_chembl_id target_uniprot_id  label  \
0     CHEMBL1000            O15245      0   
1     CHEMBL1000            P08183      1   
2     CHEMBL1000            P35367      1   
3     CHEMBL1000            Q02763      0   
4     CHEMBL1000            Q12809      0   

                                        smiles  \
0  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
1  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
2  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
3  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
4  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   

                                            sequence  \
0  MPTVDDILEQVGESGWFQKQAFLILCLLSAAFAPICVGIVFLGFTP...   
1  MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...   
2  MSLPNSSCLLEDKMCEGNKTTMASPQLMPLVVVLSTICLVTVGLNL...   
3  MDSLASLVLCGVSLLLSGTVEGAMDLILINSLPLVSDAETSLTCIA...   
4 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SMILES2Vec-style encoder: character-level tokenizer + embedding + CNN + BiGRU
Outputs fixed-size vectors for each SMILES string.

Usage (CLI):
    python smiles2vec_encoder.py \
        --input data.csv --smiles-col smiles \
        --output embeddings.parquet \
        --batch-size 256 --device cuda \
        --embedding-dim 32 --cnn-ch 64 --rnn-hidden 128 --proj-dim 256

Module usage:
    from smiles2vec_encoder import Smiles2Vec, encode_smiles
    model = Smiles2Vec(vocab=DEFAULT_VOCAB).to("cuda")
    model.eval()
    vecs, ids = encode_smiles(["CCO", "c1ccccc1"], model, device="cuda")
"""

In [2]:
from __future__ import annotations
import argparse
import math
import os
from dataclasses import dataclass
from typing import List, Tuple, Dict, Iterable, Optional

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
# -----------------------------
# 1 Tokenizer / Vocabulary
# -----------------------------
# Covers common SMILES charset; extend if your data needs more symbols
DEFAULT_VOCAB = list(
    "#%()+-./0123456789=@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]abcdefghijklmnopqrstuvwxyz|*"
)
# Special tokens
PAD = "<pad>"
UNK = "<unk>"
SOS = "<s>"
EOS = "</s>"

def build_vocab(custom: Optional[List[str]] = None) -> Tuple[Dict[str,int], List[str]]:
    base = list(dict.fromkeys((custom or []) + DEFAULT_VOCAB))  # stable, unique
    tokens = [PAD, UNK, SOS, EOS] + base
    stoi = {t:i for i,t in enumerate(tokens)}
    return stoi, tokens

@dataclass
class SmilesTokenizer:
    stoi: Dict[str, int]
    itos: List[str]
    pad_idx: int
    unk_idx: int
    sos_idx: int
    eos_idx: int
    max_len: int

    @classmethod
    def make(cls, max_len: int = 256, custom_vocab: Optional[List[str]] = None):
        stoi, itos = build_vocab(custom_vocab)
        return cls(
            stoi=stoi,
            itos=itos,
            pad_idx=stoi[PAD],
            unk_idx=stoi[UNK],
            sos_idx=stoi[SOS],
            eos_idx=stoi[EOS],
            max_len=max_len,
        )

    def encode(self, s: str) -> List[int]:
        # Truncate to leave room for SOS/EOS
        inner_max = self.max_len - 2
        s = s if len(s) <= inner_max else s[:inner_max]
        ids = [self.sos_idx]
        for ch in s:
            ids.append(self.stoi.get(ch, self.unk_idx))
        ids.append(self.eos_idx)
        # Pad
        if len(ids) < self.max_len:
            ids += [self.pad_idx] * (self.max_len - len(ids))
        return ids

    def batch_encode(self, smiles: List[str]) -> torch.LongTensor:
        arr = [self.encode(s) for s in smiles]
        return torch.tensor(arr, dtype=torch.long)


# -----------------------------
# 2 Dataset / Collate
# -----------------------------
class SmilesDataset(Dataset):
    def __init__(self, smiles: List[str], ids: Optional[List[str]] = None):
        self.smiles = smiles
        self.ids = ids if ids is not None else [str(i) for i in range(len(smiles))]

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        return {"id": self.ids[idx], "smiles": self.smiles[idx]}

def make_collate(tokenizer: SmilesTokenizer):
    def collate(batch):
        ids = [b["id"] for b in batch]
        s_list = [b["smiles"] for b in batch]
        x = tokenizer.batch_encode(s_list)  # (B, L)
        return ids, x
    return collate


# -----------------------------
# 3 SMILES2Vec Model
# -----------------------------
class Smiles2Vec(nn.Module):
    """
    Char Embedding -> 1D CNN (narrow) + GeLU -> BiGRU -> Global MaxPool -> Projection
    Returns a fixed-size vector per SMILES.
    """
    def __init__(
        self,
        vocab: List[str],
        embedding_dim: int = 32,
        cnn_channels: int = 64,
        cnn_kernel: int = 5,
        rnn_hidden: int = 128,
        rnn_layers: int = 1,
        proj_dim: int = 256,
        pad_idx: int = 0,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(len(vocab), embedding_dim, padding_idx=pad_idx)
        self.conv = nn.Conv1d(embedding_dim, cnn_channels, kernel_size=cnn_kernel, padding=cnn_kernel//2)
        self.act = nn.GELU()
        self.bigru = nn.GRU(
            input_size=cnn_channels,
            hidden_size=rnn_hidden,
            num_layers=rnn_layers,
            batch_first=True,
            bidirectional=True,
        )
        self.dropout = nn.Dropout(dropout)
        self.proj = nn.Sequential(
            nn.Linear(2*rnn_hidden, 2*rnn_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2*rnn_hidden, proj_dim),
        )
        self.norm = nn.LayerNorm(proj_dim)

        # Kaiming/Xavier sensible defaults
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.kaiming_uniform_(self.conv.weight, nonlinearity="linear")
        if self.conv.bias is not None:
            nn.init.zeros_(self.conv.bias)

    @torch.no_grad()
    def infer(self, x: torch.LongTensor) -> torch.Tensor:
        self.eval()
        return self.forward(x)

    def forward(self, x: torch.LongTensor) -> torch.Tensor:
        """
        x: (B, L) int64
        returns: (B, D) float32
        """
        # Mask for pooling
        mask = (x != self.pad_idx)  # (B, L)

        emb = self.embedding(x)              # (B, L, E)
        conv_in = emb.transpose(1, 2)        # (B, E, L)
        h = self.conv(conv_in)               # (B, C, L)
        h = self.act(h).transpose(1, 2)      # (B, L, C)
        h, _ = self.bigru(h)                 # (B, L, 2H)

        # masked global max pool over L
        h_masked = h.masked_fill(~mask.unsqueeze(-1), float("-inf"))
        pooled, _ = torch.max(h_masked, dim=1)  # (B, 2H)

        z = self.proj(self.dropout(pooled))  # (B, D)
        z = self.norm(z)
        return z


# -----------------------------
# 4) Encoding Helpers
# -----------------------------
@torch.no_grad()
def encode_loader(
    loader: DataLoader,
    model: Smiles2Vec,
    device: str = "cpu",
    normalize: bool = False,
) -> Tuple[np.ndarray, List[str]]:
    model.eval()
    all_vecs = []
    all_ids = []
    for ids, x in loader:
        x = x.to(device)
        vecs = model.infer(x)  # (B, D)
        if normalize:
            vecs = torch.nn.functional.normalize(vecs, p=2, dim=1)
        all_vecs.append(vecs.cpu().numpy())
        all_ids.extend(ids)
    return np.concatenate(all_vecs, axis=0), all_ids

@torch.no_grad()
def encode_smiles(
    smiles: List[str],
    model: Smiles2Vec,
    batch_size: int = 256,
    max_len: int = 256,
    device: str = "cpu",
    normalize: bool = False,
    ids: Optional[List[str]] = None,
) -> Tuple[np.ndarray, List[str]]:
    tokenizer = SmilesTokenizer.make(max_len=max_len)
    ds = SmilesDataset(smiles, ids=ids)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False,
                    collate_fn=make_collate(tokenizer), num_workers=0, pin_memory=(device.startswith("cuda")))
    return encode_loader(dl, model.to(device), device=device, normalize=normalize)

In [4]:
# Initialize the Smiles2Vec model and tokenizer
import torch

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Create tokenizer with default vocabulary
tokenizer = SmilesTokenizer.make(max_len=256)
print(f"Vocabulary size: {len(tokenizer.itos)}")

# Initialize the Smiles2Vec model with default parameters
model = Smiles2Vec(
    vocab=tokenizer.itos,
    embedding_dim=32,
    cnn_channels=64,
    cnn_kernel=5,
    rnn_hidden=128,
    rnn_layers=1,
    proj_dim=256,
    pad_idx=tokenizer.pad_idx,
    dropout=0.1,
)

# Move model to device and set to evaluation mode
model = model.to(device)
model.eval()
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print("Model initialized successfully!")

Using device: cpu
Vocabulary size: 81
Model parameters: 293,984
Model initialized successfully!


In [5]:
# Extract unique SMILES strings and their IDs
print("Extracting unique SMILES strings...")

# Get unique SMILES and their corresponding drug_chembl_ids
unique_smiles_df = df[['drug_chembl_id', 'smiles']].drop_duplicates(subset=['smiles'])
print(f"Total unique SMILES: {len(unique_smiles_df)}")

# Extract lists for encoding
smiles_list = unique_smiles_df['smiles'].tolist()
drug_ids = unique_smiles_df['drug_chembl_id'].tolist()

print(f"SMILES to encode: {len(smiles_list)}")
print("Sample SMILES:")
for i, smi in enumerate(smiles_list[:5]):
    print(f"  {i}: {smi[:60]}...")  # Show first 60 characters

Extracting unique SMILES strings...
Total unique SMILES: 1028
SMILES to encode: 1028
Sample SMILES:
  0: O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1...
  1: CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1...
  2: CN(C)CCOC(C)(c1ccccc1)c1ccccn1...
  3: CCC(=O)N(c1ccccc1)C1(C(=O)OC)CCN(CCC(=O)OC)CC1...
  4: N[C@@H](Cc1ccc(O)c(O)c1)C(=O)O...


In [6]:
# Encode SMILES using Smiles2Vec
from datetime import datetime
import torch

print(f"Starting encoding at {datetime.now().strftime('%H:%M:%S')}")
print(f"Encoding {len(smiles_list)} unique SMILES strings...")

# Use the encode_smiles function with batch processing
batch_size = 256  # Adjust based on your GPU memory
normalize = False  # Set to True if you want L2-normalized vectors

# Encode all SMILES
embeddings, encoded_ids = encode_smiles(
    smiles=smiles_list,
    model=model,
    batch_size=batch_size,
    max_len=256,
    device=device,
    normalize=normalize,
    ids=drug_ids
)

print(f"Encoding completed at {datetime.now().strftime('%H:%M:%S')}")
print(f"Embeddings shape: {embeddings.shape}")
print(f"Sample embedding (first 10 dimensions): {embeddings[0][:10]}")
print(f"Embedding vector length: {embeddings.shape[1]}")

Starting encoding at 19:00:19
Encoding 1028 unique SMILES strings...
Encoding completed at 19:00:25
Embeddings shape: (1028, 256)
Sample embedding (first 10 dimensions): [-1.5950437  -1.81818     1.5632088   0.17663527 -1.269098    0.07423878
  0.09562396  0.3082791   0.06107569 -0.6813537 ]
Embedding vector length: 256
Encoding completed at 19:00:25
Embeddings shape: (1028, 256)
Sample embedding (first 10 dimensions): [-1.5950437  -1.81818     1.5632088   0.17663527 -1.269098    0.07423878
  0.09562396  0.3082791   0.06107569 -0.6813537 ]
Embedding vector length: 256


In [9]:
# Save embeddings to parquet file
output_path = "smiles_embeddings_smiles2vec.parquet"

# Create DataFrame with embeddings
embeddings_df = pd.DataFrame({
    'drug_chembl_id': encoded_ids,
    'smiles': smiles_list,
    'embedding': [emb.tolist() for emb in embeddings],  # Convert numpy arrays to lists
    'embedding_dim': [embeddings.shape[1]] * len(encoded_ids)
})

# Save to parquet
embeddings_df.to_parquet(output_path, index=False)

print(f"Embeddings saved to: {output_path}")
print(f"Total embeddings saved: {len(embeddings_df)}")
print(f"Embedding dimension: {embeddings.shape[1]}")
print("\nSaved DataFrame info:")
print(embeddings_df.info())
print("\nSample saved data:")
print(embeddings_df.head(3))

Embeddings saved to: smiles_embeddings_smiles2vec.parquet
Total embeddings saved: 1028
Embedding dimension: 256

Saved DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   drug_chembl_id  1028 non-null   object
 1   smiles          1028 non-null   object
 2   embedding       1028 non-null   object
 3   embedding_dim   1028 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 32.3+ KB
None

Sample saved data:
  drug_chembl_id                                       smiles  \
0     CHEMBL1000  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1   
1     CHEMBL1002            CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1   
2     CHEMBL1004               CN(C)CCOC(C)(c1ccccc1)c1ccccn1   

                                           embedding  embedding_dim  
0  [-1.063607931137085, -0.598328173160553, 0.926...            256  
1  [-1.00967895984

# Embedding Quality Assessment

Let's validate the quality and correctness of our generated embeddings through several tests.

In [7]:
# 1. Basic Embedding Statistics and Properties
print("=== Basic Embedding Analysis ===")
print(f"Embedding shape: {embeddings.shape}")
print(f"Embedding dtype: {embeddings.dtype}")

# Check for NaN or infinite values
nan_count = np.isnan(embeddings).sum()
inf_count = np.isinf(embeddings).sum()
print(f"NaN values: {nan_count}")
print(f"Infinite values: {inf_count}")

# Statistical properties
print(f"\nEmbedding statistics:")
print(f"Mean: {embeddings.mean():.4f}")
print(f"Std: {embeddings.std():.4f}")
print(f"Min: {embeddings.min():.4f}")
print(f"Max: {embeddings.max():.4f}")

# Check if embeddings are normalized
norms = np.linalg.norm(embeddings, axis=1)
print(f"\nVector norms:")
print(f"Mean norm: {norms.mean():.4f}")
print(f"Std norm: {norms.std():.4f}")
print(f"Min norm: {norms.min():.4f}")
print(f"Max norm: {norms.max():.4f}")

print(f"Are embeddings unit normalized? {np.allclose(norms, 1.0)}")

=== Basic Embedding Analysis ===
Embedding shape: (1028, 256)
Embedding dtype: float32
NaN values: 0
Infinite values: 0

Embedding statistics:
Mean: -0.0000
Std: 0.9969
Min: -2.3843
Max: 2.7254

Vector norms:
Mean norm: 15.9496
Std norm: 0.0005
Min norm: 15.9464
Max norm: 15.9509
Are embeddings unit normalized? False


In [8]:
# 2. Test Embedding Consistency (Same SMILES should produce same embeddings)
from sklearn.metrics.pairwise import cosine_similarity

print("=== Embedding Consistency Test ===")

# Test with a few sample SMILES
test_smiles = ["CCO", "CCO", "c1ccccc1", "c1ccccc1", "CC(=O)O"]
test_embeddings, _ = encode_smiles(
    smiles=test_smiles,
    model=model,
    batch_size=32,
    max_len=256,
    device=device,
    normalize=False,
    ids=None
)

print("Test SMILES:", test_smiles)
print("Embedding shapes:", test_embeddings.shape)

# Check if identical SMILES produce identical embeddings
similarity_01 = cosine_similarity([test_embeddings[0]], [test_embeddings[1]])[0][0]
similarity_23 = cosine_similarity([test_embeddings[2]], [test_embeddings[3]])[0][0]

print(f"Cosine similarity between identical SMILES 'CCO': {similarity_01:.6f}")
print(f"Cosine similarity between identical SMILES 'c1ccccc1': {similarity_23:.6f}")

# These should be 1.0 (or very close) for identical SMILES
print(f"Are identical SMILES producing identical embeddings? {similarity_01 > 0.999 and similarity_23 > 0.999}")

=== Embedding Consistency Test ===
Test SMILES: ['CCO', 'CCO', 'c1ccccc1', 'c1ccccc1', 'CC(=O)O']
Embedding shapes: (5, 256)
Cosine similarity between identical SMILES 'CCO': 1.000000
Cosine similarity between identical SMILES 'c1ccccc1': 1.000000
Are identical SMILES producing identical embeddings? True


In [9]:
# 3. Test Chemical Similarity (Similar molecules should have similar embeddings)
print("=== Chemical Similarity Test ===")

# Define chemically similar and dissimilar pairs
similar_pairs = [
    ("CCO", "CCC"),  # Ethanol vs Propane (short alkanes)
    ("c1ccccc1", "c1ccccc1C"),  # Benzene vs Toluene (aromatic)
    ("CC(=O)O", "CCC(=O)O"),  # Acetic acid vs Propionic acid
]

dissimilar_pairs = [
    ("CCO", "c1ccccc1"),  # Alcohol vs Aromatic
    ("CC(=O)O", "CCCCCCCCCCCCCCCC"),  # Acid vs Long alkane
    ("CN(C)C", "c1ccccc1"),  # Amine vs Aromatic
]

print("Testing similar molecule pairs:")
for smi1, smi2 in similar_pairs:
    emb1, _ = encode_smiles([smi1], model, device=device)
    emb2, _ = encode_smiles([smi2], model, device=device)
    sim = cosine_similarity([emb1[0]], [emb2[0]])[0][0]
    print(f"  {smi1} vs {smi2}: {sim:.4f}")

print("\nTesting dissimilar molecule pairs:")
for smi1, smi2 in dissimilar_pairs:
    emb1, _ = encode_smiles([smi1], model, device=device)
    emb2, _ = encode_smiles([smi2], model, device=device)
    sim = cosine_similarity([emb1[0]], [emb2[0]])[0][0]
    print(f"  {smi1} vs {smi2}: {sim:.4f}")

print("\nExpected: Similar molecules should have higher similarity scores than dissimilar ones.")

=== Chemical Similarity Test ===
Testing similar molecule pairs:
  CCO vs CCC: 0.9994
  c1ccccc1 vs c1ccccc1C: 0.9997
  CC(=O)O vs CCC(=O)O: 0.9998

Testing dissimilar molecule pairs:
  CCO vs c1ccccc1: 0.9967
  CC(=O)O vs CCCCCCCCCCCCCCCC: 0.9973
  CC(=O)O vs CCC(=O)O: 0.9998

Testing dissimilar molecule pairs:
  CCO vs c1ccccc1: 0.9967
  CC(=O)O vs CCCCCCCCCCCCCCCC: 0.9973
  CN(C)C vs c1ccccc1: 0.9971

Expected: Similar molecules should have higher similarity scores than dissimilar ones.
  CN(C)C vs c1ccccc1: 0.9971

Expected: Similar molecules should have higher similarity scores than dissimilar ones.


In [10]:
# 4. Dimensionality and Uniqueness Analysis
print("=== Dimensionality and Uniqueness Analysis ===")

# Check for duplicate embeddings (which shouldn't exist for unique SMILES)
unique_embeddings = np.unique(embeddings, axis=0)
print(f"Total embeddings: {len(embeddings)}")
print(f"Unique embeddings: {len(unique_embeddings)}")
print(f"Are all embeddings unique? {len(embeddings) == len(unique_embeddings)}")

# Analyze embedding distribution
print(f"\nEmbedding variance per dimension:")
dim_variances = np.var(embeddings, axis=0)
print(f"Mean variance: {dim_variances.mean():.4f}")
print(f"Min variance: {dim_variances.min():.4f}")
print(f"Max variance: {dim_variances.max():.4f}")

# Check if any dimensions are constant (bad sign)
constant_dims = np.sum(dim_variances < 1e-6)
print(f"Dimensions with near-zero variance: {constant_dims}")

# Calculate pairwise distances to check for clustering
from sklearn.metrics.pairwise import euclidean_distances
sample_size = min(100, len(embeddings))  # Sample for efficiency
sample_embeddings = embeddings[:sample_size]
distances = euclidean_distances(sample_embeddings)

# Get upper triangle (excluding diagonal)
upper_triangle = distances[np.triu_indices_from(distances, k=1)]
print(f"\nPairwise distance statistics (sample of {sample_size}):")
print(f"Mean distance: {upper_triangle.mean():.4f}")
print(f"Std distance: {upper_triangle.std():.4f}")
print(f"Min distance: {upper_triangle.min():.4f}")
print(f"Max distance: {upper_triangle.max():.4f}")

=== Dimensionality and Uniqueness Analysis ===
Total embeddings: 1028
Unique embeddings: 1028
Are all embeddings unique? True

Embedding variance per dimension:
Mean variance: 0.0016
Min variance: 0.0006
Max variance: 0.0060
Dimensions with near-zero variance: 0

Pairwise distance statistics (sample of 100):
Mean distance: 0.9001
Std distance: 0.3499
Min distance: 0.2306
Max distance: 3.0900


In [11]:
# 5. Model Architecture Validation
print("=== Model Architecture Validation ===")

print("Current model architecture:")
print(f"- Vocabulary size: {len(tokenizer.itos)}")
print(f"- Max sequence length: {tokenizer.max_len}")
print(f"- Embedding dimension: {model.embedding.embedding_dim}")
print(f"- CNN channels: {model.conv.out_channels}")
print(f"- CNN kernel size: {model.conv.kernel_size[0]}")
print(f"- RNN hidden size: {model.bigru.hidden_size}")
print(f"- RNN layers: {model.bigru.num_layers}")
print(f"- Bidirectional RNN: {model.bigru.bidirectional}")
print(f"- Final projection dimension: {model.proj[-1].out_features}")

print(f"\nModel parameter count: {sum(p.numel() for p in model.parameters()):,}")

# Check model mode
print(f"Model in training mode: {model.training}")

print("\n" + "="*50)
print("SUMMARY AND RECOMMENDATIONS:")
print("="*50)

print("\n🔍 ABOUT YOUR CURRENT MODEL:")
print("- This is NOT the original Smiles2Vec from the research paper")
print("- This is a custom character-level CNN+RNN encoder")
print("- The original Smiles2Vec uses substructure tokenization and skip-gram training")

print("\n✅ TO VERIFY EMBEDDING QUALITY:")
print("1. Run the validation cells above")
print("2. Check that identical SMILES produce identical embeddings")
print("3. Verify that similar molecules have higher similarity scores")
print("4. Ensure no NaN or infinite values exist")
print("5. Confirm embeddings have reasonable variance across dimensions")

print("\n💡 FOR TRUE SMILES2VEC:")
print("- Consider using pre-trained models from papers like:")
print("  • 'Molecular representation learning with language models and domain-relevant auxiliary tasks'")
print("  • 'SMILES-BERT: Large Scale Unsupervised Pre-Training for Molecular Property Prediction'")
print("- Or use established libraries like RDKit descriptors or Morgan fingerprints")

print("\n🎯 YOUR CURRENT EMBEDDINGS ARE VALID IF:")
print("- They pass the consistency tests above")
print("- They show reasonable chemical similarity patterns")
print("- They work well in downstream DTI prediction tasks")

=== Model Architecture Validation ===
Current model architecture:
- Vocabulary size: 81
- Max sequence length: 256
- Embedding dimension: 32
- CNN channels: 64
- CNN kernel size: 5
- RNN hidden size: 128
- RNN layers: 1
- Bidirectional RNN: True
- Final projection dimension: 256

Model parameter count: 293,984
Model in training mode: False

SUMMARY AND RECOMMENDATIONS:

🔍 ABOUT YOUR CURRENT MODEL:
- This is NOT the original Smiles2Vec from the research paper
- This is a custom character-level CNN+RNN encoder
- The original Smiles2Vec uses substructure tokenization and skip-gram training

✅ TO VERIFY EMBEDDING QUALITY:
1. Run the validation cells above
2. Check that identical SMILES produce identical embeddings
3. Verify that similar molecules have higher similarity scores
4. Ensure no NaN or infinite values exist
5. Confirm embeddings have reasonable variance across dimensions

💡 FOR TRUE SMILES2VEC:
- Consider using pre-trained models from papers like:
  • 'Molecular representation le